### Algoritmo Genetico: Função seno 

Exemplo $f(x) = sen(10x)+1$


**Observação:**
* Utiliza a técnica de elitismo considerando os $k$ últimos melhores indivíduos da população.

In [37]:
# genetic algorithm search for continuous function optimization
from numpy.random import randint
from numpy.random import rand
from math import * 

In [38]:
# objective function
def objective(n_bits, bitstring):  
  # convert bitstring to a string of chars
  chars = ''.join([str(s) for s in bitstring])     
  # convert string to integer (decimal base)
  integer = int(chars, 2)
  x = -1 + (integer/(pow(2,n_bits)-1))*(2+1)
  y = x*sin((10*pi*x))+1            
  return y

In [39]:
# tournament selection
def selection(pop, k):
	# first random selection
	selection_ix = randint(len(pop))
	for ix in randint(0, len(pop), k-1):
		# check if better (e.g. perform a tournament)
		if pop[ix][1] > pop[selection_ix][1]:
			selection_ix = ix
	return pop[selection_ix][0]

In [40]:
# crossover two parents to create two children
def crossover(p1, p2, r_cross):
	# children are copies of parents by default
	c1, c2 = p1.copy(), p2.copy()
	# check for recombination
	if rand() < r_cross:
		# select crossover point that is not on the end of the string
		pt = randint(1, len(p1)-1)		
		# perform crossover
		c1 = p1[:pt] + p2[pt:]
		c2 = p2[:pt] + p1[pt:]
	return [c1, c2]

In [41]:
# mutation operator
def mutation(bitstring, r_mut):
	for i in range(len(bitstring)):
		# check for a mutation
		if rand() < r_mut:
			# flip the bit
			bitstring[i] = 1 - bitstring[i]

In [42]:
# apply elitism with numeber of solutions > 1
def elitism(pop, best_ind, sizeBits):
  # evaluate all candidates in the new population
  scores = [objective(sizeBits, p) for p in pop]
	# Create a temp Pop with updated fitness
  newPop = []  
  for i in zip(pop,scores):
      newPop.append(list(i))

  #Concatenate Pop with best individuals saved before
  newPop = newPop + best_ind
  # Sort from best to worst
  newPop.sort(key = lambda newPop: newPop[:][1], reverse = True)
  # Remove the worst solutions from newPop
  newPop = newPop[:-(len(best_ind))]
  # Return only bits (without fitness) >> Pattern of encoding 
  pop.clear() #Use the same data structure
  for i in newPop:
    pop.append(i[0])  
  return pop

In [43]:
# Save in a new list (copyBest) the sizeInd best individuals
def saveBestInd(pop,sizeInd):
  copyBest = []
  pop.sort(key = lambda pop: pop[:][1], reverse=True)
  copyBest = pop[:sizeInd].copy()
  return copyBest

In [44]:
def bestSolution(pop):
    pop.sort(key = lambda pop: pop[:][1], reverse=True)
    return pop[0][0],pop[0][1]

In [45]:
# genetic algorithm procedure
def genetic_algorithm(n_bits, n_iter, n_pop, r_cross, r_mut, k, sizeBestInd, optimalSol):
	# Pop: data structure used:
	##### index [0] -> number of bits (0,1)
	##### index [1] -> fitness
	pop = []
	# eltism procedure
	bestInd = []
  # initial temp population with random bitstring
	tempPop = [randint(0, 2, n_bits).tolist() for _ in range(n_pop)]	
	# enumerate generations
	for gen in range(n_iter):
			# evaluate all candidates in the population
			scores = [objective(n_bits, p) for p in tempPop]
			# Concatenate with pop
			for i in zip(tempPop,scores):
				pop.append(list(i))          
		  # keep track of best solution
			ind, best_eval = bestSolution(pop)
			print("Geração: ",gen," Ind: ",ind," Fit: ",best_eval)
			# Check if optimal is find
			if best_eval >= optimalSol:
				print(">>>>>> Melhor solução geração: %d" % gen)
				print(">>>>>> ",ind," - ",best_eval)
				break			
			# Apply elitism (Save the k best individuals)
			bestInd = saveBestInd(pop, sizeBestInd)					
			# select parents
			selected = [selection(pop, k) for _ in range(n_pop)]
			# create the next generation
			children = []
			# Current pop will be empty
			pop.clear()
			for i in range(0, n_pop, 2):
				# get selected parents in pairs
				p1, p2 = selected[i], selected[i+1]
				# crossover and mutation
				for c in crossover(p1, p2, r_cross):
					# mutation
					mutation(c, r_mut)
					# store for next generation
					children.append(c)
			# replace population
			tempPop = children
			# Update the final population with elitism
			tempPop = elitism(tempPop, bestInd, n_bits)			
	return tempPop

In [50]:
# define the total iterations
n_iter = 50
# bits per variable
n_bits = 22
# define the population size
n_pop = 4
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 0.2
# tornament selection k
k = 3
# optimal x value
x = 2.85027
# elistism size
elt = 1
# perform the genetic algorithm search
pop = genetic_algorithm(n_bits, n_iter, n_pop, r_cross, r_mut, k, elt,x)
# evaluate all candidates in the population
scores = [objective(n_bits, p) for p in pop]
# Concatenate pop and scores
print("População Final:")
for i in zip(pop,scores):
    print(i)

Geração:  0  Ind:  [1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0]  Fit:  2.8473916477781724
Geração:  1  Ind:  [1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0]  Fit:  2.8473916477781724
Geração:  2  Ind:  [1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0]  Fit:  2.8473916477781724
Geração:  3  Ind:  [1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0]  Fit:  2.8473916477781724
Geração:  4  Ind:  [1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0]  Fit:  2.8473916477781724
Geração:  5  Ind:  [1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0]  Fit:  2.8473916477781724
Geração:  6  Ind:  [1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0]  Fit:  2.8473916477781724
Geração:  7  Ind:  [1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0]  Fit:  2.8475563982552057
Geração:  8  Ind:  [1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0]  Fit:  2.847556398